In [40]:
from azureml.core import Workspace, Experiment, Dataset
from azureml.train.estimator import Estimator

ws = Workspace.from_config()

In [2]:
ENVIRONMENT_NAME = 'lightgbm-cli'
lgbm_env = ws.environments[ENVIRONMENT_NAME]

In [57]:
training_cluster = ws.compute_targets['training-cluster']
valid_ds = ws.datasets['renewal_test_csv']
train_ds = ws.datasets['renewal_train_csv']

In [31]:
print(f'docker run -it -v {train_context.mount_point}:/train -v {valid_context.mount_point}:/valid -v $(pwd):/lgbm amlcontainer /bin/bash')

# print(valid_context.mount_point)
# print(train_context.mount_point)

docker run -it -v /tmp/tmpf98z_47m:/train -v /tmp/tmp8vby3qpx:/valid -v $(pwd):/lgbm amlcontainer /bin/bash


In [58]:
parameters = {
    '--train_data_directory': train_ds.as_named_input('train').as_mount(),
    '--valid_data_directory': valid_ds.as_named_input('valid').as_mount(),
    '--task': 'train',
    '--conf_file': 'train.conf'
}

lgbm_est = Estimator(source_directory='../code/',
                     entry_script='train.py', 
                     script_params=parameters,
                     compute_target=training_cluster,
                     environment_definition=lgbm_env
                     )

In [59]:
experiment = Experiment(ws, 'lightgbm-parallel')
run = experiment.submit(lgbm_est)

In [ ]:
run.wait_for_completion(show_output=True)

RunId: lightgbm-parallel_1586296032_0af44de8
Web View: https://ml.azure.com/experiments/lightgbm-parallel/runs/lightgbm-parallel_1586296032_0af44de8?wsid=/subscriptions/dcdc374c-3ce4-4e43-92ad-10070b3b2941/resourcegroups/smart-assistant-ds/workspaces/smart-assistant-ws
